In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from scipy import stats
import requests
from requests import get
from bs4 import BeautifulSoup
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.decomposition import PCA
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import prince
import mca

In [2]:
df_acc = pd.read_csv('accessories.csv', header=None, sep='\n')
df_acc = df_acc[0].str.split(',', expand=True)
df_acc.drop(0, axis=1, inplace=True)

dict={1:'gender', 
      2:  'acc_1',
      3:'acc_2', 
      4: 'acc_3',
      5:'acc_4',
      6: 'acc_5',
      7:'acc_6',
      8:'acc_7'}
df_acc.rename(columns=dict,
          inplace=True)

df_acc = df_acc.replace(np.nan, 'No Acc', regex=True)

df_acc.head()

,gender,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7
0,Female,Green Eye Shadow,Earring,Blonde Bob,No Acc,No Acc,No Acc,No Acc
1,Male,Smile,Mohawk,No Acc,No Acc,No Acc,No Acc,No Acc
2,Female,Wild Hair,No Acc,No Acc,No Acc,No Acc,No Acc,No Acc
3,Male,Wild Hair,Nerd Glasses,Pipe,No Acc,No Acc,No Acc,No Acc
4,Male,Big Shades,Wild Hair,Earring,Goat,No Acc,No Acc,No Acc


In [3]:
types = df_acc['gender']
df_acc = df_acc.drop(['gender'], axis=1)
types

0       Female
1         Male
2       Female
3         Male
4         Male
         ...  
9995    Female
9996      Male
9997    Zombie
9998    Female
9999    Female
Name: gender, Length: 10000, dtype: object

In [5]:
mca_acc = prince.MCA(
     n_components=8,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42
 )
mca_acc = mca_acc.fit(df_acc)
mca_acc.explained_inertia_

[0.007509089898989499,
 0.006748976900211136,
 0.0065679582385858165,
 0.006185764831915067,
 0.005950862987505485,
 0.005676707578924224,
 0.0053538651833317935,
 0.00506825851079288]

In [6]:
mca_acc = mca_acc.transform(df_acc)
mca_acc = pd.DataFrame(mca_acc)
mca_acc.index.name='punk_id'
mca_acc

,0,1,2,3,4,5,6,7
punk_id,,,,,,,,
0,-0.076417,0.028294,-0.440116,-0.075440,-0.097368,-0.055712,-0.395356,0.185526
1,-0.126492,-0.353663,-0.366276,0.136356,0.201148,0.436363,0.438310,-0.080589
2,-0.129045,-0.406796,-0.297195,0.166734,0.180185,0.450648,0.246961,-0.454997
3,-0.083537,-0.496150,0.226585,0.215659,0.267193,0.043051,0.360828,0.323927
4,0.068404,0.088218,0.500622,-0.125195,-0.119704,-0.504800,-0.650615,-0.271781
...,...,...,...,...,...,...,...,...
9995,-0.104110,-0.070034,-0.623496,-0.042995,-0.065849,0.280894,-0.328502,-0.301321
9996,0.177677,-0.135079,1.126612,-0.023415,0.058632,-0.690696,-0.268341,0.413765
9997,0.022196,-0.123984,0.403228,-0.032743,-0.054065,0.019956,-0.175784,-0.065427


# Simple Linear Regression

In [8]:
df = pd.read_csv('avg_price_reg.csv', index_col = 0)
cols = df[df.columns[2:91]]
df = df.drop(cols, axis=1)
df = df.drop(['amount_in_eth'], axis=1)
df = pd.merge(df, mca_acc, on="punk_id", how='left')
df

,amount_in_dol,Alien,Ape,Female,Male,Zombie,0,1,2,3,4,5,6,7
punk_id,,,,,,,,,,,,,,
0,35933.818182,0,0,1,0,0,-0.076417,0.028294,-0.440116,-0.075440,-0.097368,-0.055712,-0.395356,0.185526
1,8037.466667,0,0,0,1,0,-0.126492,-0.353663,-0.366276,0.136356,0.201148,0.436363,0.438310,-0.080589
2,2023.214286,0,0,1,0,0,-0.129045,-0.406796,-0.297195,0.166734,0.180185,0.450648,0.246961,-0.454997
3,1304.272727,0,0,0,1,0,-0.083537,-0.496150,0.226585,0.215659,0.267193,0.043051,0.360828,0.323927
4,12092.166667,0,0,0,1,0,0.068404,0.088218,0.500622,-0.125195,-0.119704,-0.504800,-0.650615,-0.271781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,44053.333333,0,0,1,0,0,-0.104110,-0.070034,-0.623496,-0.042995,-0.065849,0.280894,-0.328502,-0.301321
9996,716.500000,0,0,0,1,0,0.177677,-0.135079,1.126612,-0.023415,0.058632,-0.690696,-0.268341,0.413765
9997,142668.818182,0,0,0,0,1,0.022196,-0.123984,0.403228,-0.032743,-0.054065,0.019956,-0.175784,-0.065427


In [11]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]
X.head()

,Alien,Ape,Female,Male,Zombie,0,1,2,3,4,5,6,7
punk_id,,,,,,,,,,,,,
0,0,0,1,0,0,-0.076417,0.028294,-0.440116,-0.075440,-0.097368,-0.055712,-0.395356,0.185526
1,0,0,0,1,0,-0.126492,-0.353663,-0.366276,0.136356,0.201148,0.436363,0.438310,-0.080589
2,0,0,1,0,0,-0.129045,-0.406796,-0.297195,0.166734,0.180185,0.450648,0.246961,-0.454997
3,0,0,0,1,0,-0.083537,-0.496150,0.226585,0.215659,0.267193,0.043051,0.360828,0.323927
4,0,0,0,1,0,0.068404,0.088218,0.500622,-0.125195,-0.119704,-0.504800,-0.650615,-0.271781


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print('Adjusted Rsquared: {}.'.format(1 - (1-regressor.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

Adjusted Rsquared: 0.09021320569339375.


In [14]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
Alien,208780.644875
Ape,-21977.447081
Female,-81055.636928
Male,-82885.538945
Zombie,-22862.021921
0,1054.277213
1,-188.140300
2,2241.737710
3,-112.947256
4,295.934110


In [15]:
y_pred = regressor.predict(X_test)
df_check = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 19049.947481750427
Mean Squared Error: 1009650118.1656134
Root Mean Squared Error: 31774.99202463493


# Panel Data Model

In [16]:
df_tr = pd.read_csv('transaction.csv', header = None, index_col = 0)
df_tr.columns = ['type', 'from', 'to', 'date', 'punk_id', 'amount_in_eth', 'amount_in_dol']
df_tr = df_tr[ ['punk_id'] + [ col for col in df_tr.columns if col != 'punk_id' ] ]
df_tr.head()

,punk_id,type,from,to,date,amount_in_eth,amount_in_dol
0,,,,,,,
0,0,Bid Withdrawn,0x983ace,NaN,2021-05-10,250.0,1030000.0
1,0,Bid,0x983ace,NaN,2021-04-11,250.0,"537,615"
2,0,Bid Withdrawn,0xd7510a,NaN,2021-03-03,100.0,"160,973"
3,0,Bid,0xd7510a,NaN,2021-02-20,100.0,"188,897"
4,0,Bid Withdrawn,natealex,NaN,2020-09-16,69.0,"25,532"


In [17]:
def fix_df(x):
    x['punk_id'] = x['punk_id'].astype('category')
    x['type'] = x['type'].astype('category')
    x['from'] = x['from'].astype(str)
    x['to'] = x['to'].astype(str)
    x['date'] = pd.to_datetime(x['date'], format = '%Y.%m.%d')
    
    x['amount_in_eth'] = x['amount_in_eth'].astype(str)
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Y', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Z', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('M', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('T', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('B', '')
    x['amount_in_eth'] = x['amount_in_eth'].replace(',', '', regex=True)
    x['amount_in_eth'] = x['amount_in_eth'].astype(float)

    x['amount_in_dol'] = x['amount_in_dol'].astype(str)
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Y', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('B', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('T', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('P', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Z', '')
    x['amount_in_dol'] = x['amount_in_dol'].replace('<', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].replace(',', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].astype(float)
    
    x['amount_in_eth'] = x['amount_in_eth'].fillna(0)
    x['amount_in_dol'] = x['amount_in_dol'].fillna(0)

    return x

In [18]:
df_tr = fix_df(df_tr)
sales = df_tr[df_tr['type'].str.contains("Sold")].reset_index(drop=True)
sales = sales.drop(['from', 'to', 'type', 'amount_in_eth'], axis=1)

In [19]:
X_panel = X.copy()
X_panel.reset_index(level=0, inplace=True)
X_panel

,punk_id,Alien,Ape,Female,Male,Zombie,0,1,2,3,4,5,6,7
0,0,0,0,1,0,0,-0.076417,0.028294,-0.440116,-0.075440,-0.097368,-0.055712,-0.395356,0.185526
1,1,0,0,0,1,0,-0.126492,-0.353663,-0.366276,0.136356,0.201148,0.436363,0.438310,-0.080589
2,2,0,0,1,0,0,-0.129045,-0.406796,-0.297195,0.166734,0.180185,0.450648,0.246961,-0.454997
3,3,0,0,0,1,0,-0.083537,-0.496150,0.226585,0.215659,0.267193,0.043051,0.360828,0.323927
4,4,0,0,0,1,0,0.068404,0.088218,0.500622,-0.125195,-0.119704,-0.504800,-0.650615,-0.271781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,1,0,0,-0.104110,-0.070034,-0.623496,-0.042995,-0.065849,0.280894,-0.328502,-0.301321
9996,9996,0,0,0,1,0,0.177677,-0.135079,1.126612,-0.023415,0.058632,-0.690696,-0.268341,0.413765
9997,9997,0,0,0,0,1,0.022196,-0.123984,0.403228,-0.032743,-0.054065,0.019956,-0.175784,-0.065427
9998,9998,0,0,1,0,0,-0.004370,0.554762,-0.410467,-0.180980,-0.200958,-0.251250,0.362564,0.422221


In [20]:
df_panel = pd.merge(sales, X_panel, on="punk_id", how='left')
df_panel = df_panel.set_index(['punk_id', 'date'])
df_panel = df_panel.fillna(0)
df_panel

amount_in_dol  Alien  Ape  Female  Male  Zombie         0  \
punk_id date                                                                    
0       2018-11-30         2822.0      0    0       1     0       0 -0.076417   
        2017-07-07          386.0      0    0       1     0       0 -0.076417   
        2017-06-23          320.0      0    0       1     0       0 -0.076417   
1       2020-11-30        36305.0      0    0       0     1       0 -0.126492   
        2019-04-06         5155.0      0    0       0     1       0 -0.126492   
...                           ...    ...  ...     ...   ...     ...       ...   
9990    2021-03-20        44981.0      0    0       0     1       0 -0.110406   
9997    2021-02-08       169770.0      0    0       0     0       1  0.022196   
        2020-09-27        21062.0      0    0       0     0       1  0.022196   
9998    2021-03-03        61170.0      0    0       1     0       0 -0.004370   
        2020-12-27         9499.0      0    0       1     0       0 -0.004370   

                           1         2         3         4         5  \
punk_id date                                                           
0       2018-11-30  0.028294 -0.440116 -0.075440 -0.097368 -0.055712   
        2017-07-07  0.028294 -0.440116 -0.075440 -0.097368 -0.055712   
        2017-06-23  0.028294 -0.440116 -0.075440 -0.097368 -0.055712   
1       2020-11-30 -0.353663 -0.366276  0.136356  0.201148  0.436363   
        2019-04-06 -0.353663 -0.366276  0.136356  0.201148  0.436363   
...                      ...       ...       ...       ...       ...   
9990    2021-03-20 -0.203904 -0.421132  0.053719  0.092300  0.282883   
9997    2021-02-08 -0.123984  0.403228 -0.032743 -0.054065  0.019956   
        2020-09-27 -0.123984  0.403228 -0.032743 -0.054065  0.019956   
9998    2021-03-03  0.554762 -0.410467 -0.180980 -0.200958 -0.251250   
        2020-12-27  0.554762 -0.410467 -0.180980 -0.200958 -0.251250   

                           6         7  
punk_id date                            
0       2018-11-30 -0.395356  0.185526  
        2017-07-07 -0.395356  0.185526  
        2017-06-23 -0.395356  0.185526  
1       2020-11-30  0.438310 -0.080589  
        2019-04-06  0.438310 -0.080589  
...                      ...       ...  
9990    2021-03-20  0.193028  0.038755  
9997    2021-02-08 -0.175784 -0.065427  
        2020-09-27 -0.175784 -0.065427  
9998    2021-03-03  0.362564  0.422221  
        2020-12-27  0.362564  0.422221  

[13531 rows x 14 columns]

In [21]:
y = df_panel['amount_in_dol']
y = y.round(decimals=-2)
X = df_panel.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [22]:
exog = sm.tools.tools.add_constant(X_train.loc[:, X_train.columns != 'Male']) #To get a full rank matrix
exog = exog.astype(int)
endog = y_train
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:          amount_in_dol   R-squared:                        0.1580
Estimator:              RandomEffects   R-squared (Between):              0.1163
No. Observations:                9471   R-squared (Within):               0.0000
Date:                Sun, Jun 27 2021   R-squared (Overall):              0.1580
Time:                        23:02:59   Log-likelihood                -1.212e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      147.88
Entities:                        5591   P-value                           0.0000
Avg Obs:                       1.6940   Distribution:                 F(12,9458)
Min Obs:                       0.0000                                           
Max Obs:                       13.000   F-statistic (robust):             147.88
                            

In [23]:
y_pred = re_res.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 79829.43715822759
Mean Squared Error: 30368214266.331944
Root Mean Squared Error: 174264.78205974936
